In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import tensorflow as tf
import numpy as np

SEED = 12011997
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    datagen = ImageDataGenerator(rotation_range=15,
                                        width_shift_range=15,
                                        height_shift_range=15,
                                        zoom_range=0.15,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.2)
else:
    datagen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
#valid_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Here of course the script expects you to have the dataset zipped in the drive folder
# As I later used flow_from_directory method, I already mapped in the correct folders the images of the training set, the script is in the attachments

!unzip '/content/drive/My Drive/datasets/MaskDataset.zip'

In [ ]:
dataset_dir = os.path.join(cwd, '')

In [ ]:
# Batch size
bs = 12

# Dimensions of images
img_h = 384
img_w = 384

num_classes = 3

classes = None

# Training
training_dir = os.path.join(dataset_dir, 'training')

#datagen = datagen = ImageDataGenerator(validation_split=0.3, rescale=1./255)

train_gen = datagen.flow_from_directory(training_dir,
                                        subset='training',
                                        target_size=(img_h, img_w),
                                        batch_size=bs,
                                        classes=classes,
                                        class_mode='categorical',
                                        shuffle=True,
                                        seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
valid_gen = datagen.flow_from_directory(training_dir,
                                        subset='validation',
                                        target_size=(img_w, img_w),
                                        batch_size=bs, 
                                        classes=classes,
                                        class_mode='categorical',
                                        shuffle=False,
                                        seed=SEED)

In [ ]:
# Training dataset
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

# Validation dataset
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()


In [ ]:
# Architecture: Features extraction -> Classifier

input_shape = [img_h, img_w, 3]
kernel_size = (3, 3)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, kernel_size=kernel_size, activation='relu' ,padding='same',input_shape=input_shape),
    tf.keras.layers.Conv2D(8, kernel_size=kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Conv2D(16, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Conv2D(16, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(32, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Conv2D(32, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.Conv2D(64, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


In [ ]:
model.summary()

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/My\ Drive/Keras3/project_CNN/

In [ ]:
import os
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras3/', 'project_CNN')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True,
                                                   save_best_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
    callbacks.append(es_callback)

In [ ]:
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)